In [1]:
import sys
import pandas
import numpy as np
import imblearn
import sklearn
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB


from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
# pandas.set_option('display.max_columns', None)
# pandas.set_option('display.max_rows', None)

### Read CSV

In [2]:
df = pandas.read_csv("ufcdata/data.csv").drop(columns = ["Referee", "R_fighter", "B_fighter", "location", "date"]).dropna().reset_index()
df = df.drop(columns = ["index"])
df.title_bout = df.title_bout.astype(int)

df["Winner"][df["Winner"] == "Red"] = 1
df["Winner"][df["Winner"] == "Blue"] = 2
df["Winner"][df["Winner"] == "Draw"] = 3


df = df[df["Winner"] != "Draw"].reset_index(drop=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [3]:
df["Winner"].value_counts()

1    2023
2    1141
3      51
Name: Winner, dtype: int64

### EDA Work

In [4]:
# list(df.keys())
R_status = []
B_status = []
global_status = []
for i in list(df.keys()):
    if("R_" in i[:3]):
        R_status.append(i)
    elif("B_" in i):
        B_status.append(i)
    else:
        global_status.append(i)
B_status.remove("B_Stance")
R_status.remove("R_Stance")

len(R_status), len(B_status)

(67, 67)

In [5]:
len(global_status)
global_status

['Winner', 'title_bout', 'weight_class', 'no_of_rounds']

In [6]:
# R_ and B_ status
df1 = pandas.DataFrame()
for side in ["R_", "B_"]:
    keys = list(sorted(df[side+"Stance"].unique()))
    lis = list()
    for i in df[side+"Stance"].tolist():
        zero = np.zeros((1,len(keys)))
        zero[0][keys.index(i)] = 1
        lis.append(zero[0])
    _keys = []
    for i in keys:
        _keys.append(side+i)
    df_temp = pandas.DataFrame(lis, columns = _keys)
    
    for i in df_temp.keys():
        df1[i] = df_temp[i]
df1.tail()

,R_Open Stance,R_Orthodox,R_Southpaw,R_Switch,B_Open Stance,B_Orthodox,B_Southpaw,B_Switch
3210,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3211,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3212,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3213,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3214,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Weight class feature
keys = list(sorted(df["weight_class"].unique()))
lis = list()
for i in df["weight_class"].tolist():
    zero = np.zeros((1,len(keys)))
    zero[0][keys.index(i)] = 1
    lis.append(zero[0])
_keys = []
for i in keys:
    _keys.append(i)
df2 = pandas.DataFrame(lis, columns = _keys).astype('float')
# df2.reset_index(drop=True)
df2.tail()

,Bantamweight,Catch Weight,Featherweight,Flyweight,Heavyweight,Light Heavyweight,Lightweight,Middleweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight
3210,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3212,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3214,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Combine three df
temp_df = df

drop_list = ["R_Stance", "B_Stance", "weight_class"]
df = (pandas.concat([df, df1,df2], axis=1)).drop(columns = drop_list)
df = df.astype('float')
df


,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,Flyweight,Heavyweight,Light Heavyweight,Lightweight,Middleweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight
0,1.0,1.0,5.0,0.0,4.0,0.0,9.200000,6.000000,0.200000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,5.0,0.0,3.0,0.0,14.600000,9.100000,11.800000,7.300000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,3.0,0.0,3.0,0.0,15.354839,11.322581,6.741935,4.387097,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,3.0,0.0,4.0,0.0,17.000000,14.000000,13.750000,11.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0.0,3.0,0.0,1.0,0.0,17.000000,14.500000,2.500000,2.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,3.0,0.0,3.0,0.0,15.000000,7.416667,6.083333,3.416667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,0.0,3.0,1.0,0.0,0.0,16.250000,11.000000,17.500000,11.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,2.0,0.0,3.0,0.0,1.0,0.0,7.250000,4.750000,1.750000,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,3.0,0.0,1.0,0.0,25.400000,17.900000,22.500000,16.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,2.0,0.0,3.0,1.0,0.0,0.0,5.428571,4.142857,12.285714,9.142857,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# The differeance between two fighter
new_key = []
for i in B_status:
    new_key.append(i.replace("B_", "Difference"))
new_key

df3 = pandas.DataFrame(df[B_status].to_numpy() - df[R_status].to_numpy(), columns = new_key)
df3.tail()


,Differencecurrent_lose_streak,Differencecurrent_win_streak,Differencedraw,Differenceavg_BODY_att,Differenceavg_BODY_landed,Differenceavg_CLINCH_att,Differenceavg_CLINCH_landed,Differenceavg_DISTANCE_att,Differenceavg_DISTANCE_landed,Differenceavg_GROUND_att,...,Differencewin_by_Decision_Split,Differencewin_by_Decision_Unanimous,Differencewin_by_KO/TKO,Differencewin_by_Submission,Differencewin_by_TKO_Doctor_Stoppage,Differencewins,DifferenceHeight_cms,DifferenceReach_cms,DifferenceWeight_lbs,Differenceage
3210,0.0,1.0,0.0,5.523810,5.238095,5.714286,5.714286,-1.619048,2.285714,-10.619048,...,0.0,-1.0,0.0,0.0,-1.0,-2.0,-7.62,0.00,-20.0,4.0
3211,0.0,0.0,0.0,8.333333,8.000000,-10.666667,-6.333333,-22.000000,-8.666667,44.666667,...,0.0,0.0,0.0,0.0,-1.0,-1.0,-12.70,-5.08,42.0,-1.0
3212,-1.0,1.0,0.0,4.400000,4.900000,8.700000,10.200000,-11.600000,-2.900000,-27.100000,...,0.0,0.0,-1.0,0.0,-1.0,-2.0,-10.16,0.00,0.0,-2.0
3213,-2.0,1.0,0.0,6.500000,4.000000,33.500000,19.500000,45.500000,18.000000,4.000000,...,0.0,1.0,0.0,0.0,0.0,1.0,2.54,5.08,20.0,6.0
3214,0.0,1.0,0.0,0.166667,0.166667,8.666667,4.000000,2.666667,1.666667,0.333333,...,0.0,0.0,2.0,-1.0,0.0,1.0,-2.54,-2.54,-20.0,-14.0


In [10]:
# Combine df as well
df = pandas.concat([df, df3], axis=1)
df.tail()

,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,Differencewin_by_Decision_Split,Differencewin_by_Decision_Unanimous,Differencewin_by_KO/TKO,Differencewin_by_Submission,Differencewin_by_TKO_Doctor_Stoppage,Differencewins,DifferenceHeight_cms,DifferenceReach_cms,DifferenceWeight_lbs,Differenceage
3210,1.0,1.0,5.0,0.0,3.0,0.0,7.666667,6.666667,10.000000,7.0,...,0.0,-1.0,0.0,0.0,-1.0,-2.0,-7.62,0.00,-20.0,4.0
3211,1.0,0.0,3.0,0.0,1.0,0.0,12.000000,10.000000,6.000000,3.0,...,0.0,0.0,0.0,0.0,-1.0,-1.0,-12.70,-5.08,42.0,-1.0
3212,1.0,1.0,5.0,0.0,1.0,0.0,7.000000,6.500000,14.500000,12.0,...,0.0,0.0,-1.0,0.0,-1.0,-2.0,-10.16,0.00,0.0,-2.0
3213,1.0,0.0,2.0,0.0,1.0,0.0,9.000000,6.000000,36.000000,21.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2.54,5.08,20.0,6.0
3214,1.0,0.0,2.0,0.0,3.0,0.0,0.666667,0.666667,8.666667,4.0,...,0.0,0.0,2.0,-1.0,0.0,1.0,-2.54,-2.54,-20.0,-14.0


In [11]:
Y = df["Winner"].tolist()

In [12]:
# scale data 
scale_df = df.drop(columns = ["Winner"])
keys = list(scale_df.keys())

scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(scale_df)
scale_df = scaler.transform(scale_df)
scale_df = pandas.DataFrame(scale_df, columns = keys)
df = scale_df


In [13]:
df["Winner"] = Y

In [14]:
df["Winner"].value_counts()

1.0    2023
2.0    1141
3.0      51
Name: Winner, dtype: int64

### Sampling the dataset

In [ ]:
X = df.drop(columns = ["Winner"])

rus = imblearn.over_sampling.SMOTE(ratio={3:100 }, random_state=1)

X_rus, Y_rus = rus.fit_sample(X, Y)

print(Counter(Y_rus))


In [24]:
# over_sampling.
X = df.drop(columns = ["Winner"])

rus = imblearn.over_sampling.ADASYN(ratio={3:500 }, random_state=1)

X_rus, Y_rus = rus.fit_sample(X, Y)

print(Counter(Y_rus))


Counter({1.0: 2023, 2.0: 1141, 3.0: 498})
(3662, 224)


In [25]:
sampled_df = pandas.DataFrame(X_rus, dtype = float, columns = df.drop(columns = ["Winner"]).keys())
sampled_df["Winner"] = Y_rus
df = sampled_df

In [ ]:
# Before Sampling

Y = df["Winner"]
PCA_df = df.drop(columns = ["Winner"])
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(PCA_df)
principalDf = pandas.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
principalDf["target"] = Y


fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0,1,3]
colors = ['r', 'g','b']
for target, color in zip(targets,colors):
    indicesToKeep = principalDf['target'] == target
    ax.scatter(principalDf.loc[indicesToKeep, 'principal component 1']
               , principalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50
               , alpha = 0.3)
ax.legend(targets)
ax.grid()

In [ ]:
candidate_components = range(10, 225, 10)
explained_ratios = []
for c in candidate_components:
    pca = PCA(n_components=c)
    X_pca = pca.fit_transform(df.drop(columns = ["Winner"]))
    explained_ratios.append(np.sum(pca.explained_variance_ratio_))

plt.figure(figsize=(10, 6), dpi=144)
plt.grid()
plt.plot(candidate_components, explained_ratios)
plt.xlabel('Number of PCA Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained variance ratio for PCA')
plt.yticks(np.arange(0.5, 1.05, .05))
plt.xticks(np.arange(0, 240, 20))

### PCA reducation 

In [ ]:
pca = PCA(n_components=120)
pca.fit(df.drop(columns = ["Winner"]))

In [ ]:
PCA_X = pca.transform(df.drop(columns = ["Winner"]))
PCA_X.shape

### Dataset 

In [26]:
# dataset = shuffle(sampled_df)
dataset = shuffle(df)



Valid_dataset = dataset[:100]
Train_dataset = dataset[100:-200]
Test_dataset = dataset[-200:]

Train_X = Train_dataset.drop(columns = ["Winner"]).to_numpy(dtype = float)
Train_Y = Train_dataset["Winner"].to_numpy(dtype = int)


Valid_X = Valid_dataset.drop(columns = ["Winner"]).to_numpy(dtype = float)
Valid_Y = Valid_dataset["Winner"].to_numpy(dtype = int)


Test_X = Test_dataset.drop(columns = ["Winner"]).to_numpy(dtype = float)
Test_Y = Test_dataset["Winner"].to_numpy(dtype = int)
pandas.DataFrame(Test_Y, columns = ["Y"])["Y"].value_counts(), 


(1    101
 2     74
 3     25
 Name: Y, dtype: int64,)

In [27]:
pandas.DataFrame(Train_Y, columns = ["Y"])["Y"].value_counts()

1    1871
2    1034
3     457
Name: Y, dtype: int64

In [28]:
pandas.DataFrame(Valid_Y, columns = ["Y"])["Y"].value_counts()

1    51
2    33
3    16
Name: Y, dtype: int64

In [ ]:
sampled_df = pandas.DataFrame(PCA_X)
sampled_df["Winner"] = Y
sampled_df
dataset = shuffle(sampled_df)



Valid_dataset = dataset[:100]
Train_dataset = dataset[100:-100]
Test_dataset = dataset[-100:]

Train_X = Train_dataset.drop(columns = ["Winner"]).to_numpy(dtype = float)
Train_Y = Train_dataset["Winner"].to_numpy(dtype = int)


Valid_X = Valid_dataset.drop(columns = ["Winner"]).to_numpy(dtype = float)
Valid_Y = Valid_dataset["Winner"].to_numpy(dtype = int)


Test_X = Valid_dataset.drop(columns = ["Winner"]).to_numpy(dtype = float)
Test_Y = Valid_dataset["Winner"].to_numpy(dtype = int)

# ML Models

### Ensembling learning method

In [29]:
# Bagging for the model and return the best selection via given datasets
rng = np.random.RandomState(1)

def Bagging(model, Train_X, Train_Y, Vail_X, Vail_Y):  
    highest = 0
    coeff = 0
    rng = np.random.RandomState(1)
    for i in range(1, 20):
        Bagging_Model = BaggingClassifier(base_estimator= model, n_estimators = i, bootstrap=True, warm_start=False) #
        Bagging_Model.fit(Train_X, Train_Y)
        curr_score = Bagging_Model.score(Vail_X, Vail_Y)
        if(highest < curr_score):
            highest = curr_score
            coeff = i
    return coeff, highest



In [30]:
# Adam boost for the model and return the best selection via given datasets
def Ada_Model(model, Train_X, Train_Y, Vail_X, Vail_Y):
    highest = 0
    coeff = 0
    rng = np.random.RandomState(1)
    for i in range(1, 20):
        AdaBoost_model = AdaBoostClassifier(base_estimator=model, n_estimators=i, random_state=rng)
        AdaBoost_model.fit(Train_X, Train_Y)
        curr_score = AdaBoost_model.score(Vail_X, Vail_Y)
        if(highest < curr_score):
            highest = curr_score
            coeff = i
    return coeff, highest



In [31]:
# GBDT(Gradient Boosting Decision Tree) for the model and return the best selection via given datasets


def GBDT(model, Train_X, Train_Y, Vail_X, Vail_Y):
    coeff = 0
    highest = 0
    _range = np.arange(0.01, 1, 0.05)
    for i in (_range):
        GBDT = GradientBoostingClassifier(init = model, learning_rate = i)
        GBDT.fit(Train_X, Train_Y)
        curr_score = GBDT.score(Vail_X, Vail_Y)
        if(highest < curr_score):
            highest = curr_score
            coeff = i
    return coeff, highest

### NB classifier

In [43]:
NB = MultinomialNB(class_prior = [1,1,0.6], alpha = 1)
NB.fit(Train_X, Train_Y)
NB.score(Train_X, Train_Y)

0.5273646638905414

In [44]:
print(sklearn.metrics.classification_report(Train_Y, NB.predict(Train_X)) )

              precision    recall  f1-score   support

           1       0.65      0.59      0.62      1871
           2       0.40      0.56      0.47      1034
           3       0.43      0.19      0.27       457

    accuracy                           0.53      3362
   macro avg       0.49      0.45      0.45      3362
weighted avg       0.54      0.53      0.52      3362



In [45]:
NB.score(Test_X, Test_Y)

0.5

In [46]:
print(sklearn.metrics.classification_report(Test_Y, NB.predict(Test_X)) )

              precision    recall  f1-score   support

           1       0.58      0.59      0.59       101
           2       0.46      0.51      0.48        74
           3       0.14      0.08      0.10        25

    accuracy                           0.50       200
   macro avg       0.39      0.40      0.39       200
weighted avg       0.48      0.50      0.49       200



In [47]:
coeff, highest = Bagging(NB, Train_X, Train_Y, Valid_X, Valid_Y)
coeff, highest

(2, 0.48)

In [48]:
Bagging_NB = BaggingClassifier(base_estimator=NB, n_estimators=coeff, random_state=rng)
Bagging_NB.fit(Train_X, Train_Y)
Bagging_NB.score(Train_X,Train_Y)

0.5181439619274242

In [49]:
print(sklearn.metrics.classification_report(Train_Y, Bagging_NB.predict(Train_X)) )

              precision    recall  f1-score   support

           1       0.66      0.56      0.61      1871
           2       0.40      0.57      0.47      1034
           3       0.33      0.22      0.26       457

    accuracy                           0.52      3362
   macro avg       0.46      0.45      0.45      3362
weighted avg       0.53      0.52      0.52      3362



In [50]:
Bagging_NB.score(Test_X, Test_Y)

0.52

In [51]:
print(sklearn.metrics.classification_report(Test_Y, Bagging_NB.predict(Test_X)) )

              precision    recall  f1-score   support

           1       0.62      0.64      0.63       101
           2       0.51      0.51      0.51        74
           3       0.05      0.04      0.04        25

    accuracy                           0.52       200
   macro avg       0.39      0.40      0.40       200
weighted avg       0.51      0.52      0.51       200



In [52]:
coeff, highest = GBDT(NB, Train_X, Train_Y, Valid_X, Valid_Y)
coeff, highest

(0.6100000000000001, 0.75)

In [53]:
GBDT_NB = GradientBoostingClassifier(init = NB, learning_rate = coeff)
GBDT_NB.fit(Train_X, Train_Y)
GBDT_NB.score(Train_X,Train_Y)

0.9982153480071386

In [54]:
print(sklearn.metrics.classification_report(Train_Y, GBDT_NB.predict(Train_X)) )

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1871
           2       1.00      0.99      1.00      1034
           3       1.00      1.00      1.00       457

    accuracy                           1.00      3362
   macro avg       1.00      1.00      1.00      3362
weighted avg       1.00      1.00      1.00      3362



In [56]:
GBDT_NB.score(Test_X,Test_Y)

0.65

In [55]:
print(sklearn.metrics.classification_report(Test_Y, GBDT_NB.predict(Test_X)) )

              precision    recall  f1-score   support

           1       0.63      0.75      0.69       101
           2       0.54      0.39      0.45        74
           3       0.96      1.00      0.98        25

    accuracy                           0.65       200
   macro avg       0.71      0.71      0.71       200
weighted avg       0.64      0.65      0.64       200



### KNN

In [73]:
KNN = KNeighborsClassifier(weights= "distance")
KNN = KNeighborsClassifier(weights= "uniform")


# neigh = KNeighborsClassifier(n_neighbors=3)
KNN.fit(Train_X, Train_Y)
KNN.score(Train_X, Train_Y)

0.7450922070196312

In [74]:
print(sklearn.metrics.classification_report(Train_Y, KNN.predict(Train_X)) )

              precision    recall  f1-score   support

           1       0.76      0.84      0.80      1871
           2       0.71      0.46      0.56      1034
           3       0.72      1.00      0.84       457

    accuracy                           0.75      3362
   macro avg       0.73      0.77      0.73      3362
weighted avg       0.74      0.75      0.73      3362



In [75]:
KNN.score(Test_X, Test_Y)

0.605

In [76]:
print(sklearn.metrics.classification_report(Test_Y, KNN.predict(Test_X)) )

              precision    recall  f1-score   support

           1       0.62      0.72      0.67       101
           2       0.52      0.31      0.39        74
           3       0.66      1.00      0.79        25

    accuracy                           0.60       200
   macro avg       0.60      0.68      0.62       200
weighted avg       0.59      0.60      0.58       200



In [77]:
coeff, highest = Bagging(KNN, Train_X, Train_Y, Valid_X, Valid_Y)
coeff, highest

(2, 0.57)

In [78]:
Bagging_KNN = BaggingClassifier(base_estimator=KNN, n_estimators=coeff, random_state=rng)
Bagging_KNN.fit(Train_X, Train_Y)
Bagging_KNN.score(Train_X,Train_Y)

0.7242712671029149

In [79]:
Bagging_KNN.score(Test_X, Test_Y)

0.62

In [80]:
print(sklearn.metrics.classification_report(Test_Y, Bagging_KNN.predict(Test_X)) )

              precision    recall  f1-score   support

           1       0.64      0.72      0.68       101
           2       0.65      0.35      0.46        74
           3       0.54      1.00      0.70        25

    accuracy                           0.62       200
   macro avg       0.61      0.69      0.61       200
weighted avg       0.63      0.62      0.60       200



In [83]:
coeff, highest = GBDT(Bagging_KNN, Train_X, Train_Y, Valid_X, Valid_Y)
coeff, highest

(0.11, 0.65)

In [85]:
GBDT_KNN = GradientBoostingClassifier(init = Bagging_KNN, learning_rate = i)
GBDT_KNN.fit(Train_X, Train_Y)
GBDT_KNN.score(Train_X,Train_Y)

TypeError: '<=' not supported between instances of 'str' and 'float'

In [ ]:
print(sklearn.metrics.classification_report(Train_Y, GBDT_KNN.predict(Train_X)) )

In [ ]:
GBDT_KNN.score(Test_X,Test_Y)

In [ ]:
print(sklearn.metrics.classification_report(Test_Y, GBDT_KNN.predict(Test_X)) )

In [ ]:
target_0 = df.loc[df['Winner'] == 0]
target_1 = df.loc[df['Winner'] == 1]
target_2 = df.loc[df['Winner'] == 3]

sns.distplot(target_0[['no_of_rounds']], hist=False, rug=True, color = "red")
sns.distplot(target_1[['no_of_rounds']], hist=False, rug=True, color = "blue")
sns.distplot(target_2[['no_of_rounds']], hist=False, rug=True, color = "green")

plt.show()

In [ ]:
df1 = pandas.DataFrame()

for side in ["R_", "B_"]:
    temp = list(sorted(df[side+"Stance"].unique()))
    
    lis = []
    for i in temp:
#         lis.append(key +" - "+i)
        lis.append(side+"Stance_"+i)
    lis = sorted(lis)
    row = np.zeros(len(lis))
    print(lis)

    onehot_df = pandas.DataFrame(columns = lis)
    row, lis
    for i in df[side+"Stance"].tolist():
        row = np.zeros(len(lis))
        index = temp.index(i)
        row[index] = 1
        row = np.reshape(row, (1, len(lis)))
        df2 = pandas.DataFrame(data = row, columns = lis)
        onehot_df = onehot_df.append(df2, ignore_index = True) 
        
    for i in onehot_df.keys():
        df1[i] = onehot_df[i].tolist()
df1.tail()